In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install SimpleITK
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 48.4 MB 24 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 5.1 MB/s 


In [ ]:
import matplotlib.pyplot as plt
import cv2
import os
import SimpleITK as sitk
from PIL import Image

In [ ]:
PATH_GT = "/content/drive/MyDrive/gt/png"  
PATH_DATA = "/content/drive/MyDrive/data/png"

In [ ]:
dirList_GT = []
for root, dirs, files in os.walk(PATH_GT):
  for dirname in dirs:
    dirList_GT.append(dirname)

In [ ]:
dirList_DATA = []
for root, dirs, files in os.walk(PATH_DATA):
  for dirname in dirs:
    dirList_DATA.append(dirname)

In [ ]:
dirList_DATA.sort()
dirList_GT.sort()


In [ ]:
PATH_GT_CROPPED = os.path.join(PATH_GT,"Cropped")
PATH_DATA_CROPPED = os.path.join(PATH_DATA,"Cropped")
# os.mkdir(PATH_GT_CROPPED)
# os.mkdir(PATH_DATA_CROPPED)

In [ ]:
def getcolordict(im):
    w,h = im.size
    colors = im.getcolors(w*h)
    colordict = { x[1]:x[0] for x in colors }
    return colordict

def percent(image):
    # image = Image.open(im)
    colordict = getcolordict(image)
    black_pixels_count = colordict.get((0,0,0))
    red_pixels_count = colordict.get((255,0,0))
    blue_pixels_count = colordict.get((0,160,255))
    yellow_pixels_count = colordict.get((255,255,128))
    if(black_pixels_count is None):
      black_pixels_count = 0
    if(blue_pixels_count is None):
      blue_pixels_count = 0
    if(red_pixels_count is None):
      red_pixels_count = 0
    if(yellow_pixels_count is None):
      yellow_pixels_count = 0
    percentage = ((red_pixels_count + yellow_pixels_count) * 100 ) / (red_pixels_count + blue_pixels_count + black_pixels_count + yellow_pixels_count)
    return percentage

In [ ]:
count_saved = 0
count_original = 0
for i in range(len(dirList_DATA)):
  path_patient_DATA = os.path.join(PATH_DATA,dirList_DATA[i])
  path_patient_GT = os.path.join(PATH_GT,dirList_GT[i])

  PATH_GT_CROPPED_PATIENT = os.path.join(PATH_GT_CROPPED,dirList_GT[i])
  os.mkdir(PATH_GT_CROPPED_PATIENT)

  PATH_DATA_CROPPED_PATIENT = os.path.join(PATH_DATA_CROPPED,dirList_DATA[i])
  os.mkdir(PATH_DATA_CROPPED_PATIENT)


  fileList_GT = []
  for root, dirs, files in os.walk(path_patient_GT):
    for filename in files:
      fileList_GT.append(filename)

  fileList_DATA = []
  for root, dirs, files in os.walk(path_patient_DATA):
    for filename in files:
      fileList_DATA.append(filename)

  fileList_GT.sort()
  fileList_DATA.sort()
  

  for i in range(len(fileList_GT)):
    filePath_GT = os.path.join(path_patient_GT,fileList_GT[i])
    filePath_DATA = os.path.join(path_patient_DATA,fileList_DATA[i])

    img_GT = Image.open(filePath_GT)
    img_DATA = Image.open(filePath_DATA)
    width, height = img_GT.size

    box = img_GT.getbbox()
    margins = (18,height-444,498,height-60)

    if box:
      count_original = count_original + 1
      img_cropped_GT = img_GT.crop(margins)
      img_cropped_DATA = img_DATA.crop(margins)

      if percent(img_cropped_GT)>=4:
        count_saved = count_saved + 1
        img_cropped_GT.save(os.path.join(PATH_GT_CROPPED_PATIENT,fileList_GT[i]))
        img_cropped_DATA.save(os.path.join(PATH_DATA_CROPPED_PATIENT,fileList_DATA[i]))
    


  


  